In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def process_code(code):
    try:
        return str(code)[:8]
    except Exception:
        return None


def sord(df):
    new_df = pd.DataFrame(columns=['time', 'code', 'IM_PQ', 'IM_Q', 'EX_PQ', 'EX_Q'])

    pd.DataFrame(columns=['time', 'code', 'IM_PQ', 'IM_Q', 'EX_PQ', 'EX_Q'])
    
    # 处理进口金额
    import_amount = df[df['指标'] == '进口金额（美元）']
    import_amount = import_amount[['时间', '商品', '数值']].rename(columns={'时间': 'time', '商品': 'code', '数值': 'IM_PQ'})
    
    # 处理进口量（仅第一数量）
    import_quantity = df[df['指标'] == '进口数量（第一数量）']
    import_quantity = import_quantity[['时间', '商品', '数值']].rename(columns={'时间': 'time', '商品': 'code', '数值': 'IM_Q'})
    
    # 处理出口金额
    export_amount = df[df['指标'] == '出口金额（美元）']
    export_amount = export_amount[['时间', '商品', '数值']].rename(columns={'时间': 'time', '商品': 'code', '数值': 'EX_PQ'})
    
    # 处理出口量（仅第一数量）
    export_quantity = df[df['指标'] == '出口数量（第一数量）']
    export_quantity = export_quantity[['时间', '商品', '数值']].rename(columns={'时间': 'time', '商品': 'code', '数值': 'EX_Q'})
    
    # 合并数据
    new_df = (
        import_amount
        .merge(export_amount, on=['time', 'code'], how='outer')
        .merge(import_quantity, on=['time', 'code'], how='outer')
        .merge(export_quantity, on=['time', 'code'], how='outer')
    )
    
    # 填充缺失值为 "空"
    new_df = new_df.fillna(pd.NA)
    new_df.head()
    return new_df



In [3]:
df = pd.read_csv('./cn.csv', encoding='gbk')[:-2]
# df['商品'] = df['商品'].str[:2]
sorted_df = sord(df)
iv = pd.read_csv('./iv.list.csv')

In [4]:
sorted_df["code"] = sorted_df["code"].str.strip().apply(lambda x: x.split()[0])

In [5]:
# 防止除以零或 NaN 的情况，计算 IM_P
sorted_df["IM_P"] = sorted_df.apply(
    lambda row: row["IM_PQ"] / row["IM_Q"] if row["IM_Q"] not in [0, np.nan] else np.nan, axis=1
)

# 防止除以零或 NaN 的情况，计算 EX_P
sorted_df["EX_P"] = sorted_df.apply(
    lambda row: row["EX_PQ"] / row["IM_Q"] if row["IM_Q"] not in [0, np.nan] else np.nan, axis=1
)

# 从 time 列提取年份并创建 year 列
sorted_df["year"] = sorted_df["time"].str[-4:].astype(int)
# sorted_df = remove2024(sorted_df)

In [6]:
sorted_df

,time,code,IM_PQ,EX_PQ,IM_Q,EX_Q,IM_P,EX_P,year
0,02-2017,01051110,NaN,NaN,NaN,NaN,NaN,NaN,2017
1,02-2017,01061110,NaN,NaN,NaN,NaN,NaN,NaN,2017
2,02-2017,01061910,1000000.0,NaN,91.0,NaN,10989.010989,NaN,2017
3,02-2017,010511,NaN,NaN,NaN,NaN,NaN,NaN,2017
4,02-2017,010611,NaN,NaN,NaN,NaN,NaN,NaN,2017
...,...,...,...,...,...,...,...,...,...
112,12-2020,010611,NaN,NaN,NaN,NaN,NaN,NaN,2020
113,12-2020,010619,100000.0,NaN,125.0,NaN,800.000000,NaN,2020
114,12-2020,0105,NaN,NaN,NaN,NaN,NaN,NaN,2020
115,12-2020,0106,100000.0,NaN,125.0,NaN,800.000000,NaN,2020


In [7]:
sorted_df['year'] = sorted_df['year'].astype(int)
iv['year'] = iv['year'].astype(int)
df = sorted_df.merge(iv, on='year', how='left')

In [8]:
df["lgdp"] = df["gdp"].apply(np.log)
df["lpop"] = df["pop"].apply(np.log)
df["lfdi"] = df["fdi"].apply(np.log)
df["lexp"] = df['EX_P'].apply(np.log)
df["limp"] = df['IM_P'].apply(np.log)
df["lexq"] = df['EX_Q'].apply(np.log)
df["limq"] = df['IM_Q'].apply(np.log)

In [10]:
df.to_csv("./01.cn.csv", encoding='utf-8', index=False)
